In [1]:
# Imports

# File handling
import os
import pathlib
from zipfile import ZipFile
import splitfolders

# Visualization
import  matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data and maths
import numpy as np
import pandas as pd
import random

# ML
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from keras.callbacks import CSVLogger # if problems with SAVING model

# Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [19]:
# image data
splitfolders.ratio("data/images_unf/", output="images", seed=586, ratio=(.8, .1, .1), group_prefix=None, move=True)

Copying files: 21165 files [01:16, 277.95 files/s]
Copying files: 0 files [42:48, ? files/s]

In [ ]:
# train, test and validation filepaths
train_path = 'images/train/'
test_path = 'images/test/'
valid_path = 'images/valid/'
size = (256, 256)
classes=['Healthy', 'Diseased']                     ############## HERE ##############

# preprocess train, test and valid data
train = image_dataset_from_directory(directory=train_path,
                         labels='inferred',
                         target_size=size,
                         color_mode="rgb",
                         batch_size=32,
                         class_mode="categorical",
                         classes=classes,
                         shuffle=True,
                         seed=42)

test = image_dataset_from_directory(directory=test_path,
                         labels='inferred',
                         target_size=size,
                         color_mode="rgb",
                         batch_size=32,
                         class_mode="categorical",
                         classes=classes,
                         shuffle=False,
                         seed=42)

valid = image_dataset_from_directory(directory=valid_path,
                         labels='inferred',
                         target_size=size,
                         color_mode="rgb",
                         batch_size=32,
                         class_mode="categorical",
                         classes=classes,
                         shuffle=True,
                         seed=42)